In [1]:
import pandas as pd
import numpy as np
import os
import json
import statistics
import pandas as pd
import numpy as np
import scipy.stats as stats
import researchpy as rp
import statsmodels.api as sm
from statsmodels.formula.api import ols
    
import matplotlib.pyplot as plt

In [3]:
d = "./result_long"

dirs_to_analyze = [os.path.join(d, o) for o in os.listdir(d) 
                    if os.path.isdir(os.path.join(d,o)) and "\\experiment" in os.path.join(d, o)]


In [59]:
rowsList = []
for curdir in dirs_to_analyze:
    row = {}
    try:
        config = json.load(open(curdir + "/config.json"))
        row['Executor Memory'] = int(config['jobClassParameters'][0]['systemParameters']['executorMemory'][:1])
        row['Batch Size'] = int(config['jobClassParameters'][0]['hyperParameters']['batchSize'])
        row['Max Epochs'] = int(config['jobClassParameters'][0]['hyperParameters']['maxEpoch'])
        row['Learning Rate'] = float(config['jobClassParameters'][0]['hyperParameters']['learningRate'])
#         row['Learning Rate Decay'] = float(config['jobClassParameters'][0]['hyperParameters']['learningrateDecay'])
        row['Nodes'] = int(config['numberOfNodes'])
        row['Lambda'] = float(config['lambda'])

    except Exception as e:
        print("Error parsing config for folder " + curdir)
        print(e)
        
    try:
        accuracy = pd.read_json(curdir + "/accuracy.json").transpose()
        row['Num finished'] = accuracy["top1"].count()
        row['Top 1 mean'] = accuracy["top1"].mean()
        row['Top 1 std'] = accuracy["top1"].std()
        row['Top 5 mean'] = accuracy["top5"].mean()
        row['Top 5 std'] = accuracy["top5"].std()
        row['loss mean'] = accuracy["loss"].mean()
        row['loss std'] = accuracy["loss"].std()
        row['lossnum'] = accuracy["lossnum"].mean()
    except Exception as e:
        print("Error parsing accuracy for folder " + curdir)
        print(e)
        
        
    try:
        time_summary = json.load(open(curdir + "/summary.json"))
        service_times = []
        process_times = []
        wait_times = []
        
        for job in time_summary['jobs']:
            service_time = float(job['end']) - float(job['arrive'])
            service_times.append(service_time)
            wait_time = float(job['start']) - float(job['arrive'])
            wait_times.append(wait_time)
            process_time = float(job['end']) - float(job['start'])
            process_times.append(process_time)

        row['Service Time mean'] = statistics.mean(service_times)
        row['Service Time std'] = statistics.stdev(service_times)
        row['Process Time mean'] = statistics.mean(process_times)
        row['Process Time std'] = statistics.stdev(process_times)
        row['Waiting Time mean'] = statistics.mean(wait_times)
        row['Waiting Time std'] = statistics.stdev(wait_times)
    except Exception as e:
        print("Error parsing time summary for folder " + curdir)
        print(e)
        
    
    rowsList.append(row) 
    
analysis = pd.DataFrame(rowsList)
analysis = analysis.drop([25, 21, 10])
analysis

,Executor Memory,Batch Size,Max Epochs,Learning Rate,Nodes,Lambda,Num finished,Top 1 mean,Top 1 std,Top 5 mean,Top 5 std,loss mean,loss std,lossnum,Service Time mean,Service Time std,Process Time mean,Process Time std,Waiting Time mean,Waiting Time std
0,1,128,10,0.005,1,0.001,18,0.917161,0.004836,0.995978,0.000658,0.311462,0.014277,157.0,919.756175,537.041835,546.247377,48.413280,373.508798,528.354603
1,1,128,20,0.005,1,0.001,18,0.937539,0.003374,0.997989,0.000327,0.221215,0.010395,157.0,2068.116207,940.238236,979.102109,71.374752,1089.014098,972.418779
2,1,128,5,0.005,1,0.001,24,0.887213,0.007875,0.992421,0.001042,0.459345,0.024686,157.0,386.533227,155.775823,312.385778,38.010766,74.147449,152.634580
3,1,256,10,0.005,1,0.001,16,0.881700,0.010511,0.991550,0.001534,0.475815,0.027349,79.0,416.221998,121.978029,374.529568,35.783015,41.692430,112.565025
4,1,256,20,0.005,1,0.001,18,0.918178,0.004882,0.996078,0.000634,0.306919,0.018671,79.0,1004.807165,369.385723,669.282307,40.808168,335.524858,371.995803
5,1,256,5,0.005,1,0.001,14,0.823957,0.015476,0.985393,0.001936,0.780267,0.050272,79.0,276.894724,113.805099,233.967428,32.571365,42.927296,117.037215
6,1,64,10,0.005,1,0.001,22,0.937982,0.004802,0.997768,0.000628,0.222201,0.016329,313.0,2350.760193,700.342934,840.787190,85.752967,1509.973003,717.731563
7,1,64,20,0.005,1,0.001,11,0.952736,0.003015,0.998936,0.000320,0.164343,0.008531,313.0,6582.009186,4174.904672,1607.945413,135.006008,4974.063773,4127.299052
8,1,64,5,0.005,1,0.001,19,0.916179,0.005239,0.995747,0.000617,0.314247,0.015869,313.0,699.964803,287.617910,484.069147,51.983902,215.895656,293.119735
9,1,128,10,0.010,1,0.001,15,0.944153,0.002963,0.998247,0.000412,0.195491,0.009902,157.0,658.405138,214.577698,552.444823,40.014074,105.960315,200.308006


In [61]:
# Process time results:
print("Max avg process time: ", analysis["Process Time mean"].max())
print("Min avg process time: ", analysis["Process Time mean"].min())
print("Mean avg process time: ", analysis["Process Time mean"].mean())

print("Min mu: ", "%.7f" % (1/analysis["Process Time mean"].max()))
print("Max mu: ", "%.7f" % (1/analysis["Process Time mean"].min()))
print("Mean mu: ", "%.7f" % (1/analysis["Process Time mean"].mean()))


Max avg process time:  1607.9454131343148
Min avg process time:  221.33136041959128
Mean avg process time:  624.6510084900796
Min mu:  0.0006219
Max mu:  0.0045181
Mean mu:  0.0016009


In [62]:
important_cols=[x for x in analysis.columns if "Top 1" in x]
important_cols.extend([x for x in analysis.columns if "Time" in x])


In [63]:
# Executor Memory
from IPython.display import HTML, display

to_show = analysis.columns[0:5]
for col in to_show:
    display(rp.summary_cont(analysis[important_cols].groupby(analysis[col])))

Top 1 mean                                                    \
                         N      Mean        SD        SE 95% Conf.  Interval   
Executor Memory                                                                
1                       24  0.933425  0.035949  0.007338  0.919043  0.947808   

                Top 1 std                                ...  \
                        N      Mean        SD        SE  ...   
Executor Memory                                          ...   
1                      24  0.004757  0.003123  0.000637  ...   

                Waiting Time mean                                      \
                               SD          SE  95% Conf.     Interval   
Executor Memory                                                         
1                     1279.800068  261.238095  217.13121  1241.184542   

                Waiting Time std                                      \
                               N        Mean          SD          SE   
Executor Memory                                                        
1                             24  636.607961  955.415211  195.023313   

                                          
                  95% Conf.     Interval  
Executor Memory                           
1                254.362267  1018.853655  

[1 rows x 48 columns]

Top 1 mean                                                    \
                    N      Mean        SD        SE 95% Conf.  Interval   
Batch Size                                                                
64                  8  0.952913  0.019480  0.006887  0.939415  0.966412   
128                 8  0.937754  0.028381  0.010034  0.918087  0.957421   
256                 8  0.909608  0.044245  0.015643  0.878948  0.940269   

           Top 1 std                                ... Waiting Time mean  \
                   N      Mean        SD        SE  ...                SD   
Batch Size                                          ...                     
64                 8  0.003322  0.001262  0.000446  ...       1916.126489   
128                8  0.003939  0.002029  0.000717  ...        477.733198   
256                8  0.007010  0.004173  0.001475  ...        161.450299   

                                                Waiting Time std               \
                    SE   95% Conf.     Interval                N         Mean   
Batch Size                                                                      
64          677.453017  313.362830  2968.978656                8  1277.329770   
128         168.904192   56.491034   718.595467                8   416.222319   
256          57.081301   46.880285   270.638984                8   216.271794   

                                                              
                     SD          SE   95% Conf.     Interval  
Batch Size                                                    
64          1444.691475  510.775569  276.209655  2278.449886  
128          399.204334  141.140046  139.587829   692.856808  
256          161.741506   57.184258  104.190649   328.352940  

[3 rows x 48 columns]

Top 1 mean                                                    \
                    N      Mean        SD        SE 95% Conf.  Interval   
Max Epochs                                                                
5                   9  0.914280  0.042937  0.014312  0.886227  0.942332   
10                  8  0.938514  0.030624  0.010827  0.917293  0.959735   
20                  7  0.952225  0.020125  0.007607  0.937316  0.967134   

           Top 1 std                                ... Waiting Time mean  \
                   N      Mean        SD        SE  ...                SD   
Max Epochs                                          ...                     
5                  9  0.006374  0.003811  0.001270  ...         70.358775   
10                 8  0.004350  0.002850  0.001008  ...        564.081331   
20                 7  0.003142  0.001164  0.000440  ...       1973.826931   

                                                Waiting Time std               \
                    SE   95% Conf.     Interval                N         Mean   
Max Epochs                                                                      
5            23.452925   39.318071   131.253537                9   145.613577   
10          199.432867  130.096375   911.873215                8   463.115876   
20          746.036456  332.674035  3257.136943                7  1466.163123   

                                                              
                     SD          SE   95% Conf.     Interval  
Max Epochs                                                    
5             87.285664   29.095221   88.586943   202.640212  
10           355.519924  125.695275  216.753137   709.478614  
20          1461.819655  552.515896  383.231968  2549.094279  

[3 rows x 48 columns]

Top 1 mean                                                    \
                       N      Mean        SD        SE 95% Conf.  Interval   
Learning Rate                                                                
0.005                  9  0.908072  0.039074  0.013025  0.882543  0.933600   
0.010                  8  0.937716  0.025439  0.008994  0.920088  0.955344   
0.020                  7  0.961119  0.016726  0.006322  0.948728  0.973510   

              Top 1 std                                ... Waiting Time mean  \
                      N      Mean        SD        SE  ...                SD   
Learning Rate                                          ...                     
0.005                 9  0.006668  0.004032  0.001344  ...       1588.571294   
0.010                 8  0.004301  0.001644  0.000581  ...       1457.839360   
0.020                 7  0.002821  0.001555  0.000588  ...        510.247233   

                                                   Waiting Time std  \
                       SE   95% Conf.     Interval                N   
Learning Rate                                                         
0.005          529.523765  -76.005761  1999.727396                9   
0.010          515.424049 -296.596703  1723.865567                8   
0.020          192.855327   69.713018   825.705898                7   

                                                                             
                     Mean           SD          SE   95% Conf.     Interval  
Learning Rate                                                                
0.005          821.461817  1273.541481  424.513827  -10.585284  1653.508918  
0.010          587.304018   958.918559  339.028908  -77.192642  1251.800677  
0.020          455.286082   413.471766  156.277638  148.981911   761.590252  

[3 rows x 48 columns]

Top 1 mean                                                   Top 1 std  \
               N      Mean        SD        SE 95% Conf.  Interval         N   
Nodes                                                                          
1             24  0.933425  0.035949  0.007338  0.919043  0.947808        24   

                                     ... Waiting Time mean              \
           Mean        SD        SE  ...                SD          SE   
Nodes                                ...                                 
1      0.004757  0.003123  0.000637  ...       1279.800068  261.238095   

                              Waiting Time std                          \
       95% Conf.     Interval                N        Mean          SD   
Nodes                                                                    
1      217.13121  1241.184542               24  636.607961  955.415211   

                                            
               SE   95% Conf.     Interval  
Nodes                                       
1      195.023313  254.362267  1018.853655  

[1 rows x 48 columns]

In [64]:
display(rp.summary_cont(analysis[[x for x in important_cols if "Process" in x]].groupby(analysis["Batch Size"])))

Process Time mean                                                  \
                           N        Mean          SD          SE   95% Conf.   
Batch Size                                                                     
64                         8  889.145062  467.656961  165.341704  565.075322   
128                        8  559.081138  270.666061   95.694903  371.519127   
256                        8  425.726826  200.817184   70.999596  286.567617   

                        Process Time std                                   \
               Interval                N       Mean         SD         SE   
Batch Size                                                                  
64          1213.214802                8  76.700167  29.660145  10.486445   
128          746.643148                8  48.655878  15.925945   5.630672   
256          564.886035                8  35.500315   9.764795   3.452376   

                                  
            95% Conf.   Interval  
Batch Size                        
64          56.146736  97.253599  
128         37.619761  59.691995  
256         28.733657  42.266973

In [65]:
analysis.sort_values(by=['Top 1 mean'], ascending=False)

,Executor Memory,Batch Size,Max Epochs,Learning Rate,Nodes,Lambda,Num finished,Top 1 mean,Top 1 std,Top 5 mean,Top 5 std,loss mean,loss std,lossnum,Service Time mean,Service Time std,Process Time mean,Process Time std,Waiting Time mean,Waiting Time std
24,1,64,10,0.020,1,0.001,19,0.975842,0.001787,0.999842,0.000130,0.080110,0.004492,313.0,1970.924572,978.695520,793.006613,86.758666,1177.917959,1012.168823
19,1,128,20,0.020,1,0.001,17,0.975312,0.001554,0.999718,0.000167,0.082534,0.004806,157.0,2131.789124,1059.114402,940.633077,74.634033,1191.156046,1069.387943
16,1,64,20,0.010,1,0.001,12,0.969558,0.002465,0.999750,0.000178,0.101347,0.006461,313.0,5859.385485,2868.508936,1592.238778,91.269198,4267.146708,2870.756740
18,1,128,10,0.020,1,0.001,13,0.966015,0.001799,0.999608,0.000112,0.112956,0.004824,157.0,717.662456,226.052954,531.444488,46.034087,186.217967,232.688118
22,1,256,20,0.020,1,0.001,18,0.965900,0.002365,0.999528,0.000190,0.115465,0.007051,79.0,920.973086,395.503711,638.995857,49.093591,281.977229,385.722739
26,1,64,5,0.020,1,0.001,20,0.965280,0.002302,0.999535,0.000142,0.117415,0.006289,313.0,662.333662,330.421610,464.639759,50.906135,197.693903,292.583318
15,1,64,10,0.010,1,0.001,16,0.960869,0.002772,0.999294,0.000267,0.133583,0.008410,313.0,1580.519892,808.741149,843.792611,66.610356,736.727281,817.195708
7,1,64,20,0.005,1,0.001,11,0.952736,0.003015,0.998936,0.000320,0.164343,0.008531,313.0,6582.009186,4174.904672,1607.945413,135.006008,4974.063773,4127.299052
20,1,128,5,0.020,1,0.001,17,0.951624,0.004234,0.998841,0.000300,0.168360,0.012923,157.0,329.824813,77.016140,299.089247,39.693972,30.735566,72.340956
13,1,256,20,0.010,1,0.001,21,0.946352,0.004343,0.998548,0.000483,0.186291,0.013137,79.0,1085.230202,487.968889,659.774492,47.430263,425.455710,465.560808


In [66]:
analysis.sort_values(by=['Top 1 mean'], ascending=False)

,Executor Memory,Batch Size,Max Epochs,Learning Rate,Nodes,Lambda,Num finished,Top 1 mean,Top 1 std,Top 5 mean,Top 5 std,loss mean,loss std,lossnum,Service Time mean,Service Time std,Process Time mean,Process Time std,Waiting Time mean,Waiting Time std
24,1,64,10,0.020,1,0.001,19,0.975842,0.001787,0.999842,0.000130,0.080110,0.004492,313.0,1970.924572,978.695520,793.006613,86.758666,1177.917959,1012.168823
19,1,128,20,0.020,1,0.001,17,0.975312,0.001554,0.999718,0.000167,0.082534,0.004806,157.0,2131.789124,1059.114402,940.633077,74.634033,1191.156046,1069.387943
16,1,64,20,0.010,1,0.001,12,0.969558,0.002465,0.999750,0.000178,0.101347,0.006461,313.0,5859.385485,2868.508936,1592.238778,91.269198,4267.146708,2870.756740
18,1,128,10,0.020,1,0.001,13,0.966015,0.001799,0.999608,0.000112,0.112956,0.004824,157.0,717.662456,226.052954,531.444488,46.034087,186.217967,232.688118
22,1,256,20,0.020,1,0.001,18,0.965900,0.002365,0.999528,0.000190,0.115465,0.007051,79.0,920.973086,395.503711,638.995857,49.093591,281.977229,385.722739
26,1,64,5,0.020,1,0.001,20,0.965280,0.002302,0.999535,0.000142,0.117415,0.006289,313.0,662.333662,330.421610,464.639759,50.906135,197.693903,292.583318
15,1,64,10,0.010,1,0.001,16,0.960869,0.002772,0.999294,0.000267,0.133583,0.008410,313.0,1580.519892,808.741149,843.792611,66.610356,736.727281,817.195708
7,1,64,20,0.005,1,0.001,11,0.952736,0.003015,0.998936,0.000320,0.164343,0.008531,313.0,6582.009186,4174.904672,1607.945413,135.006008,4974.063773,4127.299052
20,1,128,5,0.020,1,0.001,17,0.951624,0.004234,0.998841,0.000300,0.168360,0.012923,157.0,329.824813,77.016140,299.089247,39.693972,30.735566,72.340956
13,1,256,20,0.010,1,0.001,21,0.946352,0.004343,0.998548,0.000483,0.186291,0.013137,79.0,1085.230202,487.968889,659.774492,47.430263,425.455710,465.560808


In [67]:
## Service time / accuracy^(something) (adapted IES)
alpha = 1
analysis["Acc/Service"] = analysis['Service Time mean']/(analysis['Top 1 mean']**alpha)
display(analysis.sort_values(by=['Acc/Service']).head())
display(analysis.sort_values(by=['Acc/Service']).tail())

,Executor Memory,Batch Size,Max Epochs,Learning Rate,Nodes,Lambda,Num finished,Top 1 mean,Top 1 std,Top 5 mean,...,loss mean,loss std,lossnum,Service Time mean,Service Time std,Process Time mean,Process Time std,Waiting Time mean,Waiting Time std,Acc/Service
14,1,256,5,0.010,1,0.001,20,0.888530,0.007465,0.992570,...,0.443630,0.022459,79.0,265.213771,76.943273,226.862363,21.774074,38.351408,71.266929,298.486006
23,1,256,5,0.020,1,0.001,18,0.927861,0.005703,0.996839,...,0.264007,0.016852,79.0,289.598896,132.591448,221.331360,25.969854,68.267535,122.110675,312.114488
5,1,256,5,0.005,1,0.001,14,0.823957,0.015476,0.985393,...,0.780267,0.050272,79.0,276.894724,113.805099,233.967428,32.571365,42.927296,117.037215,336.054766
20,1,128,5,0.020,1,0.001,17,0.951624,0.004234,0.998841,...,0.168360,0.012923,157.0,329.824813,77.016140,299.089247,39.693972,30.735566,72.340956,346.591694
11,1,128,5,0.010,1,0.001,27,0.923015,0.004878,0.996385,...,0.284732,0.015587,157.0,360.907965,109.594844,311.302201,31.072060,49.605764,101.645565,391.009937


,Executor Memory,Batch Size,Max Epochs,Learning Rate,Nodes,Lambda,Num finished,Top 1 mean,Top 1 std,Top 5 mean,...,loss mean,loss std,lossnum,Service Time mean,Service Time std,Process Time mean,Process Time std,Waiting Time mean,Waiting Time std,Acc/Service
19,1,128,20,0.020,1,0.001,17,0.975312,0.001554,0.999718,...,0.082534,0.004806,157.0,2131.789124,1059.114402,940.633077,74.634033,1191.156046,1069.387943,2185.751482
1,1,128,20,0.005,1,0.001,18,0.937539,0.003374,0.997989,...,0.221215,0.010395,157.0,2068.116207,940.238236,979.102109,71.374752,1089.014098,972.418779,2205.899117
6,1,64,10,0.005,1,0.001,22,0.937982,0.004802,0.997768,...,0.222201,0.016329,313.0,2350.760193,700.342934,840.787190,85.752967,1509.973003,717.731563,2506.189513
16,1,64,20,0.010,1,0.001,12,0.969558,0.002465,0.999750,...,0.101347,0.006461,313.0,5859.385485,2868.508936,1592.238778,91.269198,4267.146708,2870.756740,6043.355355
7,1,64,20,0.005,1,0.001,11,0.952736,0.003015,0.998936,...,0.164343,0.008531,313.0,6582.009186,4174.904672,1607.945413,135.006008,4974.063773,4127.299052,6908.531523


In [68]:
## LISAS score, lower is better

analysis["LISAS"] = analysis['Service Time mean'] + analysis["Service Time std"] / analysis["Top 1 std"] * (1-analysis["Top 1 mean"])
display(analysis.sort_values(by=['LISAS']).head())
display(analysis.sort_values(by=['LISAS']).tail())


,Executor Memory,Batch Size,Max Epochs,Learning Rate,Nodes,Lambda,Num finished,Top 1 mean,Top 1 std,Top 5 mean,...,loss std,lossnum,Service Time mean,Service Time std,Process Time mean,Process Time std,Waiting Time mean,Waiting Time std,Acc/Service,LISAS
20,1,128,5,0.020,1,0.001,17,0.951624,0.004234,0.998841,...,0.012923,157.0,329.824813,77.016140,299.089247,39.693972,30.735566,72.340956,346.591694,1209.722237
14,1,256,5,0.010,1,0.001,20,0.888530,0.007465,0.992570,...,0.022459,79.0,265.213771,76.943273,226.862363,21.774074,38.351408,71.266929,298.486006,1414.199206
5,1,256,5,0.005,1,0.001,14,0.823957,0.015476,0.985393,...,0.050272,79.0,276.894724,113.805099,233.967428,32.571365,42.927296,117.037215,336.054766,1571.462835
12,1,256,10,0.010,1,0.001,19,0.924389,0.005335,0.996463,...,0.016276,79.0,416.951840,86.902843,381.071232,30.572190,35.880608,83.915159,451.056455,1648.612133
3,1,256,10,0.005,1,0.001,16,0.881700,0.010511,0.991550,...,0.027349,79.0,416.221998,121.978029,374.529568,35.783015,41.692430,112.565025,472.067596,1789.052134


,Executor Memory,Batch Size,Max Epochs,Learning Rate,Nodes,Lambda,Num finished,Top 1 mean,Top 1 std,Top 5 mean,...,loss std,lossnum,Service Time mean,Service Time std,Process Time mean,Process Time std,Waiting Time mean,Waiting Time std,Acc/Service,LISAS
24,1,64,10,0.020,1,0.001,19,0.975842,0.001787,0.999842,...,0.004492,313.0,1970.924572,978.695520,793.006613,86.758666,1177.917959,1012.168823,2019.716681,15198.656356
19,1,128,20,0.020,1,0.001,17,0.975312,0.001554,0.999718,...,0.004806,157.0,2131.789124,1059.114402,940.633077,74.634033,1191.156046,1069.387943,2185.751482,18962.416706
1,1,128,20,0.005,1,0.001,18,0.937539,0.003374,0.997989,...,0.010395,157.0,2068.116207,940.238236,979.102109,71.374752,1089.014098,972.418779,2205.899117,19475.325769
16,1,64,20,0.010,1,0.001,12,0.969558,0.002465,0.999750,...,0.006461,313.0,5859.385485,2868.508936,1592.238778,91.269198,4267.146708,2870.756740,6043.355355,41286.620955
7,1,64,20,0.005,1,0.001,11,0.952736,0.003015,0.998936,...,0.008531,313.0,6582.009186,4174.904672,1607.945413,135.006008,4974.063773,4127.299052,6908.531523,72027.396118


In [75]:
## Adapted RCS
c = analysis["Num finished"] * analysis["Top 1 mean"]
sumRT = analysis["Num finished"] * analysis["Service Time mean"]
analysis["RCS"] = c / sumRT
display(analysis.sort_values(by=['RCS'], ascending=False).head())
display(analysis.sort_values(by=['RCS'], ascending=False).tail())

,Executor Memory,Batch Size,Max Epochs,Learning Rate,Nodes,Lambda,Num finished,Top 1 mean,Top 1 std,Top 5 mean,...,lossnum,Service Time mean,Service Time std,Process Time mean,Process Time std,Waiting Time mean,Waiting Time std,Acc/Service,LISAS,RCS
14,1,256,5,0.010,1,0.001,20,0.888530,0.007465,0.992570,...,79.0,265.213771,76.943273,226.862363,21.774074,38.351408,71.266929,298.486006,1414.199206,0.003350
23,1,256,5,0.020,1,0.001,18,0.927861,0.005703,0.996839,...,79.0,289.598896,132.591448,221.331360,25.969854,68.267535,122.110675,312.114488,1966.768360,0.003204
5,1,256,5,0.005,1,0.001,14,0.823957,0.015476,0.985393,...,79.0,276.894724,113.805099,233.967428,32.571365,42.927296,117.037215,336.054766,1571.462835,0.002976
20,1,128,5,0.020,1,0.001,17,0.951624,0.004234,0.998841,...,157.0,329.824813,77.016140,299.089247,39.693972,30.735566,72.340956,346.591694,1209.722237,0.002885
11,1,128,5,0.010,1,0.001,27,0.923015,0.004878,0.996385,...,157.0,360.907965,109.594844,311.302201,31.072060,49.605764,101.645565,391.009937,2090.563834,0.002557


,Executor Memory,Batch Size,Max Epochs,Learning Rate,Nodes,Lambda,Num finished,Top 1 mean,Top 1 std,Top 5 mean,...,lossnum,Service Time mean,Service Time std,Process Time mean,Process Time std,Waiting Time mean,Waiting Time std,Acc/Service,LISAS,RCS
19,1,128,20,0.020,1,0.001,17,0.975312,0.001554,0.999718,...,157.0,2131.789124,1059.114402,940.633077,74.634033,1191.156046,1069.387943,2185.751482,18962.416706,0.000458
1,1,128,20,0.005,1,0.001,18,0.937539,0.003374,0.997989,...,157.0,2068.116207,940.238236,979.102109,71.374752,1089.014098,972.418779,2205.899117,19475.325769,0.000453
6,1,64,10,0.005,1,0.001,22,0.937982,0.004802,0.997768,...,313.0,2350.760193,700.342934,840.787190,85.752967,1509.973003,717.731563,2506.189513,11396.212171,0.000399
16,1,64,20,0.010,1,0.001,12,0.969558,0.002465,0.999750,...,313.0,5859.385485,2868.508936,1592.238778,91.269198,4267.146708,2870.756740,6043.355355,41286.620955,0.000165
7,1,64,20,0.005,1,0.001,11,0.952736,0.003015,0.998936,...,313.0,6582.009186,4174.904672,1607.945413,135.006008,4974.063773,4127.299052,6908.531523,72027.396118,0.000145
